In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/misinformation-fake-news-text-dataset-79k/EXTRA_RussianPropagandaSubset.csv
/kaggle/input/misinformation-fake-news-text-dataset-79k/DataSet_Misinfo_TRUE.csv
/kaggle/input/misinformation-fake-news-text-dataset-79k/DataSet_Misinfo_FAKE.csv


In [15]:
from warnings import filterwarnings
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import re
import string

In [3]:
TrueNews = pd.read_csv("/kaggle/input/misinformation-fake-news-text-dataset-79k/DataSet_Misinfo_TRUE.csv")
FakeNews = pd.read_csv("/kaggle/input/misinformation-fake-news-text-dataset-79k/DataSet_Misinfo_FAKE.csv")

# Explore and Preprocess

In [5]:
TrueNews.head()


,Unnamed: 0,text
0,0,The head of a conservative Republican faction ...
1,1,Transgender people will be allowed for the fir...
2,2,The special counsel investigation of links bet...
3,3,Trump campaign adviser George Papadopoulos tol...
4,4,President Donald Trump called on the U.S. Post...


In [ ]:
TrueNews.columns


In [ ]:
TrueNews.shape


In [ ]:
TrueNews.isna().sum()


In [4]:
TrueNews.dropna(inplace=True)

In [5]:
TrueNews.shape

(34946, 2)

In [6]:
FakeNews.shape

(43642, 2)

In [7]:
FakeNews.dropna(inplace=True)

In [8]:
FakeNews.shape

(43642, 2)

In [9]:
nltk.download('stopwords')
from nltk.corpus import stopwords
from textblob import Word, TextBlob
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
cmd = "unzip -d /usr/share/nltk_data/corpora/ /usr/share/nltk_data/corpora/wordnet.zip"
os.system(cmd)
os.system("ls /usr/share/nltk_data/corpora/wordnet*")

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /usr/share/nltk_data...
Archive:  /usr/share/nltk_data/corpora/wordnet.zip
   creating: /usr/share/nltk_data/corpora/wordnet/
  inflating: /usr/share/nltk_data/corpora/wordnet/lexnames  
  inflating: /usr/share/nltk_data/corpora/wordnet/data.verb  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.adv  
  inflating: /usr/share/nltk_data/corpora/wordnet/adv.exc  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.verb  
  inflating: /usr/share/nltk_data/corpora/wordnet/cntlist.rev  
  inflating: /usr/share/nltk_data/corpora/wordnet/data.adj  
  inflating: /usr/share/nltk_data/corp

0

In [10]:
def lemmatize_with_postag(sentence):
    sent = TextBlob(sentence)
    tag_dict = {"J": 'a', 
                "N": 'n', 
                "V": 'v', 
                "R": 'r'}
    words_and_tags = [(w, tag_dict.get(pos[0], 'n')) for w, pos in sent.tags]    
    lemmatized_list = [wd.lemmatize(tag) for wd, tag in words_and_tags]
    return " ".join(lemmatized_list)

In [11]:
def text_preprocess(textColumn):
    textColumn = textColumn.str.lower()                         #transform to lower case
    textColumn = textColumn.str.replace('[^\w\s]','',regex=True) # remove punction
    textColumn = textColumn.str.replace('\d', '', regex=True)     #remove number
    sw = stopwords.words('english')
    textColumn = textColumn.apply(lambda x: " ".join(x for x in str(x).split() if x not in sw)) # remove stop words
    words_count = pd.Series(' '.join(textColumn).split()).value_counts()
    rare_word = words_count[words_count<=1]
    textColumn = textColumn.apply(lambda x: " ".join(x for x in x.split() if x not in rare_word)) #remove rare words
    textColumn = textColumn.apply(lambda x: lemmatize_with_postag(x)) #Lemmatization:  reduces different inflected forms of words (like 'running' to 'run') to their base form. 
    return textColumn

In [12]:
def wordopt(text):
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub("\\W"," ",text) 
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)    
    return text

In [16]:
TrueNews["text"]=TrueNews["text"].apply(wordopt)


In [18]:
FakeNews["text"]=FakeNews["text"].apply(wordopt)


In [19]:
FakeNews["class"] = 0
TrueNews["class"] = 1

In [20]:
df_merge = pd.concat([FakeNews, TrueNews], axis =0 )
df_merge.head(10)

,Unnamed: 0,text,class
0,0,donald trump just couldn t wish all americans ...,0
1,1,house intelligence committee chairman devin nu...,0
2,2,on friday it was revealed that former milwauk...,0
3,3,on christmas day donald trump announced that ...,0
4,4,pope francis used his annual christmas day mes...,0
5,5,the number of cases of cops brutalizing and ki...,0
6,6,donald trump spent a good portion of his day a...,0
7,7,in the wake of yet another court decision that...,0
8,8,many people have raised the alarm regarding th...,0
9,9,just when you might have thought we d get a br...,0


In [21]:
df_merge = df_merge.sample(frac = 1)


In [22]:
df_merge.reset_index(inplace = True)
df_merge.drop(["index"], axis = 1, inplace = True)

In [23]:
df_merge.head()


,Unnamed: 0,text,class
0,40287,a subsequent us uk and france missile strike ...,0
1,26562,on the monday edition of breitbart news daily ...,1
2,20696,a group of allies of ivory coast s former pres...,1
3,35445,home health former dea prescription head d...,0
4,18100,muslim rohingya insurgents said on saturday th...,1


# Split the data and encode with tf-idf

In [24]:
x = df_merge["text"]
y = df_merge["class"]

In [25]:
x_train, x_test_valid, y_train, y_test_valid = train_test_split(x, y, test_size=0.3)


In [26]:
x_test, x_valid, y_test, y_valid = train_test_split(x_test_valid, y_test_valid, test_size = 0.33)


In [27]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorization = TfidfVectorizer()
xv_train = vectorization.fit_transform(x_train)
xv_test = vectorization.transform(x_test)
xv_valid = vectorization.transform(x_valid)

# Logistic Regression

In [28]:
from sklearn.linear_model import LogisticRegression

LR = LogisticRegression()
LR.fit(xv_train,y_train)

LogisticRegression()

In [29]:
pred_lr=LR.predict(xv_test)


In [30]:
print(classification_report(y_test, pred_lr))


              precision    recall  f1-score   support

           0       0.93      0.95      0.94      8736
           1       0.94      0.91      0.93      7060

    accuracy                           0.93     15796
   macro avg       0.93      0.93      0.93     15796
weighted avg       0.93      0.93      0.93     15796



# Decision Tree

In [31]:
from sklearn.tree import DecisionTreeClassifier

DT = DecisionTreeClassifier()
DT.fit(xv_train, y_train)

DecisionTreeClassifier()

In [34]:
pred_dt = DT.predict(xv_test)


In [35]:
DT.score(xv_test, y_test)


0.8909217523423651

In [36]:
print(classification_report(y_test, pred_dt))


              precision    recall  f1-score   support

           0       0.89      0.91      0.90      8736
           1       0.89      0.86      0.88      7060

    accuracy                           0.89     15796
   macro avg       0.89      0.89      0.89     15796
weighted avg       0.89      0.89      0.89     15796



# GradientBoostingClassifier

In [37]:
from sklearn.ensemble import GradientBoostingClassifier

GBC = GradientBoostingClassifier(random_state=123)
GBC.fit(xv_train, y_train)

GradientBoostingClassifier(random_state=123)

In [38]:
pred_gbc = GBC.predict(xv_test)


In [39]:
GBC.score(xv_test, y_test)


0.905798936439605

In [40]:
print(classification_report(y_test, pred_gbc))


              precision    recall  f1-score   support

           0       0.90      0.93      0.92      8736
           1       0.91      0.88      0.89      7060

    accuracy                           0.91     15796
   macro avg       0.91      0.90      0.90     15796
weighted avg       0.91      0.91      0.91     15796



# Random Forest

In [41]:
from sklearn.ensemble import RandomForestClassifier

RFC = RandomForestClassifier(random_state=123)
RFC.fit(xv_train, y_train)

RandomForestClassifier(random_state=123)

In [42]:
pred_rfc = RFC.predict(xv_test)

In [43]:
RFC.score(xv_test, y_test)

0.9242846290200051

In [44]:
print(classification_report(y_test, pred_rfc))

              precision    recall  f1-score   support

           0       0.91      0.95      0.93      8736
           1       0.94      0.89      0.91      7060

    accuracy                           0.92     15796
   macro avg       0.93      0.92      0.92     15796
weighted avg       0.92      0.92      0.92     15796



# Validation

In [45]:
pred_rfc_valid = RFC.predict(xv_valid)


In [46]:
print(classification_report(y_valid, pred_rfc_valid))


              precision    recall  f1-score   support

           0       0.91      0.96      0.93      4334
           1       0.94      0.88      0.91      3447

    accuracy                           0.92      7781
   macro avg       0.93      0.92      0.92      7781
weighted avg       0.93      0.92      0.92      7781



In [47]:
pred_gbc_valid = GBC.predict(xv_valid)


In [48]:
print(classification_report(y_valid, pred_gbc_valid))


              precision    recall  f1-score   support

           0       0.91      0.93      0.92      4334
           1       0.91      0.88      0.90      3447

    accuracy                           0.91      7781
   macro avg       0.91      0.91      0.91      7781
weighted avg       0.91      0.91      0.91      7781



In [49]:
pred_DT_valid = DT.predict(xv_valid)


In [50]:
print(classification_report(y_valid, pred_DT_valid))


              precision    recall  f1-score   support

           0       0.90      0.92      0.91      4334
           1       0.90      0.87      0.88      3447

    accuracy                           0.90      7781
   macro avg       0.90      0.89      0.89      7781
weighted avg       0.90      0.90      0.90      7781



In [51]:
pred_LR_valid = LR.predict(xv_valid)


In [52]:
print(classification_report(y_valid, pred_LR_valid))


              precision    recall  f1-score   support

           0       0.93      0.95      0.94      4334
           1       0.93      0.92      0.92      3447

    accuracy                           0.93      7781
   macro avg       0.93      0.93      0.93      7781
weighted avg       0.93      0.93      0.93      7781



# Conclusion
1. Logistic regression have the best performance while other complex models are not so good.
2. Decision tree and random forest's perfomance is not good which could be a reason that tree based model is not the best option here.